In [1]:
import pandas as pd


In [3]:
df = pd.read_csv('./demo.csv', delim_whitespace=True)
from datasets import Dataset
# print(df)

dataset = Dataset.from_pandas(df)
print(dataset)


Dataset({
    features: ['concept', 'lnwordstart', 'lnwordend', 'labels'],
    num_rows: 159
})


d:\miniconda3\envs\cs7643-final-poc\lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [139]:
dataset.features

dataset = dataset.rename_column("type", "labels")

dataset.set_format(type="torch")

In [140]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  # {'concept': 'hgb', 'lnwordstart': '61:12', 'lnwordend': '61:12', 'labels': 'test'}
  return tokenizer(
      batch["concept"],
      max_length=180,
      truncation=True,
      padding="max_length"
  )

dataset = dataset.map(encode_batch, batched=True)


loading file vocab.txt from cache at C:\Users\Sarah/.cache\huggingface\hub\models--bert-base-uncased\snapshots\1dbc166cf8765166998eff31ade2eb64c8a40076\vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\Sarah/.cache\huggingface\hub\models--bert-base-uncased\snapshots\1dbc166cf8765166998eff31ade2eb64c8a40076\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\Sarah/.cache\huggingface\hub\models--bert-base-uncased\snapshots\1dbc166cf8765166998eff31ade2eb64c8a40076\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_

In [141]:
from transformers import (
    BertConfig,
    BertAdapterModel,
    BertForSequenceClassification
)

id2label = {0: 'test', 1: 'treatment', 2: 'problem'}


config = BertConfig.from_pretrained(
    "bert-base-uncased"
)

# Check how to save our own tuned model
model = BertAdapterModel.from_pretrained(
    "bert-base-uncased",
    config=config,
)


loading configuration file config.json from cache at C:\Users\Sarah/.cache\huggingface\hub\models--bert-base-uncased\snapshots\1dbc166cf8765166998eff31ade2eb64c8a40076\config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at C:\Users\Sarah/.cache\huggingface\hub\models--bert-base-uncased\snapshots\1dbc166cf8765166998eff31ade2eb64c8a40076\pytorch_model.bin
Generate conf

In [142]:
from transformers.adapters.composition import Fuse, Parallel

# Load the pre-trained adapters we want to fuse
# To check how we can use our own adapters here
model.load_adapter("nli/multinli@ukp", load_as="multinli", with_head=False)
model.load_adapter("sts/qqp@ukp", with_head=False)
model.load_adapter("nli/qnli@ukp", with_head=False)
# Add a fusion layer for all loaded adapters
model.add_adapter_fusion(Fuse("multinli", "qqp", "qnli"))
model.set_active_adapters(Fuse("multinli", "qqp", "qnli"))

# Add a classification head for our target task
model.add_classification_head("cb", num_labels=3)

Attempting to load adapter from source 'ah'...
No exactly matching adapter config found for this specifier, falling back to default.
Resolved adapter files at https://public.ukp.informatik.tu-darmstadt.de/AdapterHub/text_task/mnli/bert-base-uncased/pfeiffer/bert-base-uncased_nli_multinli_pfeiffer.zip.
Loading module configuration from ~/.cache\torch\adapters\2e2b596bb3b1b6db529d746b87272bda4c8892b0a26f6a960553852cc4378654-5ad6785a3c6c5d82b0a96c3612e27fccc2f710cd379f14326924e06e815c48eb-extracted\adapter_config.json
Adding adapter 'multinli'.
Loading module weights from ~/.cache\torch\adapters\2e2b596bb3b1b6db529d746b87272bda4c8892b0a26f6a960553852cc4378654-5ad6785a3c6c5d82b0a96c3612e27fccc2f710cd379f14326924e06e815c48eb-extracted\pytorch_adapter.bin
Some module weights could not be found in loaded weights file: invertible_adapters.multinli.F.0.weight, invertible_adapters.multinli.F.0.bias, invertible_adapters.multinli.F.2.weight, invertible_adapters.multinli.F.2.bias, invertible_adapte

In [143]:
# Unfreeze and activate fusion setup
adapter_setup = Fuse("multinli", "qqp", "qnli")
model.train_adapter_fusion(adapter_setup)

In [144]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction
from datasets import Dataset, DatasetDict

training_args = TrainingArguments(
    learning_rate=5e-5,
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

split_datatset = dataset.train_test_split(test_size=0.1)

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=split_datatset['train'],
    eval_dataset=split_datatset['test'],
    compute_metrics=compute_accuracy
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [145]:
trainer.train()

d:\miniconda3\envs\cs7643-final-poc\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 143
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 25
  Number of trainable parameters = 21844995
























































Training completed. Do not forget to share your model on huggingface.co/models =)


                                      
                                   





                    


                                      

  0%|          | 0/25 [50:29<?, ?it/s]







100%|██████████| 25/25 [31:27<00:00, 75.52s/it]

{'train_runtime': 1887.9351, 'train_samples_per_second': 0.379, 'train_steps_per_second': 0.013, 'train_loss': 0.9843036651611328, 'epoch': 5.0}


TrainOutput(global_step=25, training_loss=0.9843036651611328, metrics={'train_runtime': 1887.9351, 'train_samples_per_second': 0.379, 'train_steps_per_second': 0.013, 'train_loss': 0.9843036651611328, 'epoch': 5.0})

In [146]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 16
  Batch size = 32


100%|██████████| 1/1 [00:00<00:00, 30.31it/s]


{'eval_loss': 0.9215760827064514,
 'eval_acc': 0.75,
 'eval_runtime': 13.5158,
 'eval_samples_per_second': 1.184,
 'eval_steps_per_second': 0.074,
 'epoch': 5.0}

In [137]:
print(split_datatset['train'][0])

{'concept': 'hgb', 'lnwordstart': '61:12', 'lnwordend': '61:12', 'labels': 'test', 'input_ids': tensor([  101,  1044, 18259,   102,  3231,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,    